# SQL Server Assessment Tool

Unlike other notebooks, **do not execute all cells of this notebook!** 

A single assessment may take awhile so fill out the variables and execute the cell that matches the desired environment to perform the assessment needed. Only one of these cells needs to be executed after the variables are defined.

## Notebook Variables

| Line | Variable | Description |
| ---- | -------- | ----------- |
| 1 | ServerInstance | Name of the SQL Server instance |
| 2 | Group | (Optional) Name of the server group, if known | 

In [ ]:
$ServerInstance = ""
$Group          = ""

## Notebook Steps
1.  Ensure that the proper APIs and modules are installed per the <a href="../prereqs.ipynb">prerequisites</a> notebook
2.  Define a service instance and group corresponding to the SQL Server instances to be assessed
3.  Choose an example below that corresponds to the appropriate task
4.  Execute only that example's code block and wait for results
5.  Fix any recommended issues and rerun Assessment API until clear

### Example 1 Get-SqlInstance
Pipe the output of the Get-SqlInstance cmdlet to the _Get-SqlAssessmentItem_ cmdlet to pass the instance object to it to perform an assessment on the entire instance. 

In [ ]:
Get-SqlInstance -ServerInstance $serverInstance | Get-SqlAssessmentItem

### Example 2 - Get-Item
Get a list of available databases in the instance using the _Get-Item_ cmdlet and pipe it to _Get-SqlAssessmentItem_. 

In [ ]:
Get-Item SQLSERVER:\SQL\$serverInstance\default | Get-SqlAssessmentItem

### Example 3 - Get-SqlDatabase
Or, use the _Get-SqlDatabase_ cmdlet without specifying a name to get a full list of database objects.

In [ ]:
Get-SqlDatabase -ServerInstance $serverInstance | Get-SqlAssessmentItem

### Example 4 - Export Assessment to SQL Table
Invoke assessment for the instance and save the results to a SQL table by piping the output of the _Get-SqlInstance_ cmdlet to the _Invoke-SqlAssessment_ cmdlet. The results are piped to the _Write-SqlTableData_ cmdlet. The _Invoke-Assessment_ cmdlet is run with the -**FlattenOutput** parameter in this example. This parameter makes the output suitable for the _Write-SqlTableData_ cmdlet. The latter raises an error if the parameter is omitted.

In [ ]:
Get-SqlInstance -ServerInstance $serverInstance |
Invoke-SqlAssessment -FlattenOutput |
Write-SqlTableData -ServerInstance $serverInstance -DatabaseName SQLAssessmentDemo -SchemaName Assessment -TableName Results -Force

### Example 5 - SQL Server (SSMS) Locally Registered Servers
To gather locally saved SQL registered servers, use the SQL Server PowerShell module installed from the [Pre-requisites notebook](..\prereqs.ipynb). 

In [ ]:
Import-Module SqlServer
$RegisteredServers = Get-ChildItem SQLSERVER:\SQLRegistration -Recurse | Where-Object {$_.IsLocal -eq $True}
$RegisteredServers.Refresh() 
$RegisteredServers | Format-List -Property *

### Example 5 - Specifying a Central Management Server
Many large organizations use [Registered Servers](https://docs.microsoft.com/en-us/sql/ssms/register-servers/register-servers) to organize their networks within editors like SSMS or ADS. To take advantage of running the SQL Server Assessment API against all or a specific Registered Server Group, specify the CMS instance below. 

![](CMS.png)

Take advantage of the [dbatools CMS module](https://dbatools.io/cms/) for this task. 

In [ ]:
# Get a list of all servers stored on the CMS instance 
# Note that this list is pipable to all dbatools commands

if($SqlInstance -And $Group){
  Get-DbaRegServer -SqlInstance $SqlInstance -Group $Group
}else{
  Get-DbaRegServer -SqlInstance $SqlInstance
}